Exploring the $\chi^2$ distribution
===

Start with some model to generate random data.  <br> 
We'll use $0.2+\frac{sin(x)}{x+1}$ over the range $0\leq x<15$ <br>
Generate $N$ random numbers according to this distribution and make a histogram of the results using 30 bins. 

An easy way to generate random numbers according to an arbitrary distribution is to use rejection sampling (here's a great [decription  of the technique](https://matthewfeickert.github.io/Statistics-Notes/notebooks/simulation/Rejection-Sampling-MC.html) )

1) Make a histogram of your random distribution.  Verify that the the numbers you generated agree with the shape of the function.  Generate at least 10000 points and plot your data in a histogram with errorbars.  You can use numpy+matplotlib+etc, PyROOT, or a mix.  Think about how to appropriately normalize your function, then overlay it with the data.  The normalization depends on the number of bins, the bin range and the number of points thrown.  You may find it easier to first nromalize the area of your function to 1.0, thus turning it into a proper PDF.  

In [79]:
# your work goes here

# generate the distribution. this represents a shape, not a normalized PDF.
# the definite numerical integral of the function is
# 0.666090171898448
n_points = 10*1000 # points to draw
n_bins = 30
x_minimum = 0
x_maximum = 15
n_parameters = 4

import ROOT as R

tc1 = R.TCanvas()

# original function, not normalized
tfDistribution = R.TF1("tfDistribution", "0.2 + sin(x) / (x + 1)", x_minimum, x_maximum, 0)
area_under_curve = tfDistribution.Integral(x_minimum, x_maximum)
# print(area_under_curve)

# function to get histogram from is below!
tfDist = R.TF1("tfDist", "(1./[0]) * (0.2 + sin(x) / (x + 1))", x_minimum, x_maximum, 1) # remember 1/3 = 0, so need 1./3
tfDist.SetParameter(0, area_under_curve)
tfDist.Update()
integral = tfDist.Integral(0,15)
# print(integral)

# sample data from the distribution and draw the histogram.
h1 = R.TH1F("h1", "10,000 random data points from function", n_bins, x_minimum, x_maximum)
h1.FillRandom("tfDist", n_points)
h1.Draw()
h1.Draw("same, e") # overlay with errorbars
tc1.Draw()

# make a new canvas to overlay the distribution and the histogram together
tc2 = R.TCanvas()

# function to plot needs to scale with histogram n_points
tfDist_plotting = R.TF1("tfDist_plotting", "([1]/[0])*([2]/[3])*(0.2 + sin(x) / (x + 1))", x_minimum, x_maximum, n_parameters)
tfDist_plotting.SetParameter(0, area_under_curve)
tfDist_plotting.SetParameter(1, float(n_points))
tfDist_plotting.SetParameter(2, float(x_maximum-x_minimum))
tfDist_plotting.SetParameter(3, n_bins)
tfDist_plotting.Update()
h1.Draw()
h1.Draw('same, e')
tfDist_plotting.Draw("SAME")
tc2.Draw()

# h1.SetDirectory(R.nullptr)
# del h1

2) Repeat the above experiment at least 1000 times.  For each experiment calculate the $\chi^2$ of your data with respect to your (properly normalized) model.  Plot (histogram) the $\chi^2$ distribution and compare it to the functional form of the $\chi^2(ndof=30)$ distribution.  You can find the form of the $\chi^2(ndof)$ PDF function in many places and implement it using the Gamma function or use a premade function, eg:
* [wikipedia](https://en.wikipedia.org/wiki/Chi-squared_distribution)
* [scipy.stats.chi2](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html#scipy.stats.chi2)

In [80]:
# your work goes here
n_trials = 1000

# We have to calculate chi-square for each point in each trial.
# The chi-square we are calculating is with respect to the entries in each bin.
# We already have error bars from the histograms above, so we can use those as 
# our sigmas in the chi-square formula.
# Now, we just need to get the "ideal histogram" that has our expectation values
# for each bin.

# Retrieving the content and error from the bins is relatively straight-forward.
# We must call the methods GetBinContent() and GetBinError() on the histogram.

# loop to get the content and errors of each bin.
# should make this into a function and call it!

def calculate_bin_information(histogram):
    """
    Returns the number of bins and center, content, and error for each bin in a pyROOT histogram object.
    Skips the 0th and final bins, since they're for under- and over-flow only.
    Parameter: TH1F Histogram object
    Returns:
    data : a list containing the following in order
        count : int, number of bins in histogram
        centers : float list length of count, location of centers for each bin
        contents : float list length of count, has number in bin
        errors : float list length of count, has error for each bin
    """
    centers = []
    contents = []
    errors = []
    for i in range(1, histogram.GetNbinsX() + 1):
        centers.append(histogram.GetBinCenter(i))
        contents.append(histogram.GetBinContent(i))
        errors.append(histogram.GetBinError(i))
    count = i
    data = [count, centers, contents, errors]

    return data

# hist_data = calculate_bin_information(h1)
# print(hist_data)
        
# for i in range(1, h1.GetNbinsX() + 1):
#     bin_center = h1.GetBinCenter(i) # location
#     bin_content = h1.GetBinContent(i) # number
#     bin_error = h1.GetBinError(i) # error
#     print(f"Bin {i} (center: {bin_center:.2f}): Content = {bin_content:.3f} and Error = {bin_error}")

# Now that we have the bin centers, we should get the "ideal histogram" by taking
# the centers and plugging that in to the stretched distribution that fits the
# histogram from earlier.
# easy = tfDist_plotting(3.25)
# print(easy)

def calculate_chi_square(data, distribution):
    """
    given the data from calculate_bin_information() and distribution,
    calculates the chi-square of that particular interaction.
    """
    total_chi_square = 0.
    for i in range(data[0]): # data[0] is the number of bins
        center = data[1][i]
        measured_value = data[2][i]
        expected_value = distribution(center)
        residual = measured_value - expected_value
        sigma = data[3][i]
        partial_chi = residual / sigma
        partial_chi_square = partial_chi * partial_chi
        total_chi_square += partial_chi_square

    return total_chi_square

all_the_likes = calculate_chi_square(calculate_bin_information(h1), tfDist_plotting)
# print(all_the_likes)

# now, just loop over a temporary histogram.
# make a new one for the new chi-square data!
list_of_chi = []
for i in range(n_trials):
    h1.SetDirectory(R.nullptr)
    del h1
    h1 = R.TH1F("h1", "10,000 random data points from function", n_bins, x_minimum, x_maximum)
    h1.FillRandom("tfDist", n_points)
    chi_square = calculate_chi_square(calculate_bin_information(h1),
                                      tfDist_plotting)
    list_of_chi.append(chi_square)

# print(list_of_chi)

# Now that we have the chi-square data, let's put it into a histogram!

h2 = R.TH1F("h2", "1,000 chi-squares from earlier data", n_bins, 0, 60)
for chi in list_of_chi:
    h2.Fill(chi) # can only fill one at a time :(

# now for plotting,
tc3 = R.TCanvas()
h2.Draw()
h2.Draw('same, e')
tc3.Draw()

# but we also want the functional form of the chi-square distribution

tc4 = R.TCanvas()
h2.Draw()
h2.Draw('same, e')
from scipy.stats import chi2
import numpy as np
x = np.linspace(0, 60, n_points) # get 1000 x_values
y = chi2.pdf(30, x)

for i, element in enumerate(y):
    y[i] *= n_trials * 2
chi_distribution = R.TGraph(n_points, x, y)
chi_distribution.Draw('same')
tc4.Draw()
# print(chi2.pdf(5, 2))
# print(chi2.pdf(5, 2))

Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).


**Only required for Phys5630**

3) Modify your code above to perform the following study. <br>
Repeat (2) for nbins = $ndof = 30, 50, 75, 100$ and calculate the reduced $\chi^2$, eg $\chi^2/ndof$ for each experiment.  Plot the mean value of the reduced $\chi^2$ with errorbars versus $ndof$.  Make a table comparing your calculations to the expected results.

In [82]:
# your work goes here
# Let's just redo the calculations above for multiple numbers of bins.
# Easiest way is to just make a function for that. But, I don't know how to make that work :(

ndof = [30, 50, 75, 100]

list_of_chi = []
h2.SetDirectory(R.nullptr)
del h2
n_bins = ndof[0]
for i in range(n_trials):
    h1 = R.TH1F("h1", "10,000 random data points from function", n_bins, x_minimum, x_maximum)
    h1.FillRandom("tfDist", n_points)
    chi_square = calculate_chi_square(calculate_bin_information(h1),tfDist_plotting)
    list_of_chi.append(chi_square)
    h1.SetDirectory(R.nullptr)
    del h1
h2 = R.TH1F("h2", "1,000 chi-squares from earlier data", n_bins, 0, 200)
for chi in list_of_chi:
    h2.Fill(chi) # can only fill one at a time :(
# now for plotting,
tc4 = R.TCanvas()
h2.Draw()
h2.Draw('same, e')
from scipy.stats import chi2
import numpy as np
x = np.linspace(0, 200, n_points) # get 1000 x_values
y = chi2.pdf(n_bins, x)

for i, element in enumerate(y):
    y[i] *= n_trials * 1
chi_distribution = R.TGraph(n_points, x, y)
chi_distribution.Draw('same')
# tc4.Draw()
tc4.Update()
tc4.Draw()